In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
filename_crosswalk = {
    2023 : "23-RC-Pub-Data-Set.xlsx",
    2022 : "2022-Report-Card-Public-Data-Set.xlsx",
    2021 : "2021-RC-Pub-Data-Set.xlsx",
    2020 : "2020-Report-Card-Public-Data-Set.xlsx",
    2019 : "2019-Report-Card-Public-Data-Set.xlsx",
    2018 : "Report-Card-Public-Data-Set.xlsx",
    2017 : "rc17.txt",
    2016 : "rc16.txt",
    2015 : "rc15.txt",
    2014 : "rc14.txt",
    2013 : "rc13.txt",
    2012 : "rc12.txt",
    2011 : "rc11u.txt",
    2010 : "rc10.txt",
    2009 : "rc09.txt",
    2008 : "rc08u.txt"
}

demographic_key = {
    "Female" : "Female",
    "FEMALE" : "Female",
    "Male" : "Male",
    "MALE" : "Male",
    "White" : "White",
    "WHITE" : "White",
    "WHITE %" : "White",
    "White %" : "White",
    "Asian" : "Asian",
    "ASIAN" : "Asian",
    "ASIAN %" : "Asian",
    "Asian %" : "Asian",
    "Asian5" : "Asian",
    "Black" : "Black",
    "BLACK" : "Black",
    "BLACK %" : "Black",
    "Black or African American":"Black",
    "Black or African American %" : "Black",
    "Black or African American3" : "Black",
    "Latinx" : "Latinx",
    "HISPANIC" : "Latinx",
    "HISPANIC %" : "Latinx",
    "Hispanic or Latino":"Latinx",
    "Hispanic or Latino %" : "Latinx",
    "Hispanic" : "Latinx",
    "Hispanic or Latino4" : "Latinx",
    "American Indian or Alaska Native" : "American Indian or Alaska Native",
    "NATIVE AMERICAN" : "American Indian or Alaska Native",
    "NATIVE AMERICAN %" : "American Indian or Alaska Native",
    "American Indian or Alaska Native %" : "American Indian or Alaska Native",
    "Am Ind/Alaska Nat" : "American Indian or Alaska Native",
    "Am Ind/Alaska Nat2" : "American Indian or Alaska Native",
    "Am Ind/Alaska Nat7" : "American Indian or Alaska Native",
    "Native Hawaiian or Other Pacific Islander" : "Native Hawaiian or Other Pacific Islander",
    "NATIVE HAWAIIAN AND OTHERS" : "Native Hawaiian or Other Pacific Islander",
    "NATIVE HAWAIIAN AND OTHERS %" : "Native Hawaiian or Other Pacific Islander",
    "Native Hawaiian or Other Pacific Islander %" : "Native Hawaiian or Other Pacific Islander",
    "NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER %" : "Native Hawaiian or Other Pacific Islander",
    "NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER" : "Native Hawaiian or Other Pacific Islander",
    "HawaiiPacIslander" : "Native Hawaiian or Other Pacific Islander",
    "Hawaiian/Pac Islander" : "Native Hawaiian or Other Pacific Islander",
    "Nat Haw/Other Pac Isndr" : "Native Hawaiian or Other Pacific Islander",
    "Nat Haw/Other Pac Isndr6" : "Native Hawaiian or Other Pacific Islander",
    "Two or More Races" : "Two or More Races",
    "MULTIRACIAL" : "Two or More Races",
    "MULTIRACIAL %" : "Two or More Races",
    "MULTIRACIAL/ETHNIC %" : "Two or More Races",
    "MULTIRACIAL/ETHNIC %" : "Two or More Races",
    "MultiRace" : "Two or More Races",
    "MultiRace3" : "Two or More Races",
    "TWO OR MORE RACES" : "Two or More Races",
    "TWO OR MORE RACES %" : "Two or More Races",
    "TOW OR MORE RACES" : "Two or More Races",
    "TOW OR MORE RACES %" : "Two or More Races",
    "Two or More Races %" : "Two or More Races",
    "Two or More Race" : "Two or More Races",
    "Two or More Races8" : "Two or More Races",
    "EL" : "EL",
    "EL %" : "EL",
    "LEP" : "EL",
    "L.E.P." : "EL",
    "LEP %" : "EL",
    "Low Income" : "Low Income",
    "LOW INCOME" : "Low Income",
    "Low Income %" : "Low Income",
    "LOW INCOME %" : "Low Income",
    "Low-Income" : "Low Income",
    "LOW-INCOME" : "Low Income",
    "Low-Income %" : "Low Income",
    "LOW-INCOME %" : "Low Income",
    "LowIncome" : "Low Income",
    "Migrant" : "Migrant",
    "MIGRANT" : "Migrant",
    "MIGRANT %" : "Migrant",
    "Homeless" : "Homeless",
    "Homeless %" : "Homeless",
    "HOMELESS" : "Homeless",
    "HOMELESS %" : "Homeless",
    "IEP" : "IEP",
    "I.E.P." : "IEP",
    "IEP %" : "IEP",
    "Children with Disabilities" : "Children with Disabilities",
    "CWD" : "Children with Disabilities",
    "CWD %" : "Children with Disabilities",
}


In [3]:
def get_layout_file(short_year):
    if int(short_year) > 12:
        x = "x"
    else:
        x = ""
    if short_year == "12" or short_year == "16" or short_year == "15":
        return pd.read_excel("./data/RC" + short_year + "-layout.xls" + x, header=None)
    else:
        return pd.read_excel("./data/RC" + short_year + "_layout.xls" + x, header=None)
    
def fix_enrollment(layout_df):
    for enrollment_key in ['SCHOOL', 'SUBREGION', 'DISTRICT', 'STATE']:
        enrollment_rate_mask = layout_df['Metric'].str.startswith(enrollment_key + ' - ')
        layout_df.loc[enrollment_rate_mask,'Demographic'] = layout_df.loc[enrollment_rate_mask, 'Metric'].str.split(" - ").apply(lambda x: x[-1])
        layout_df.loc[enrollment_rate_mask, 'Metric'] = enrollment_key + ' ENROLLMENT'

        special_demos = ['L.E.P.', 'I.E.P.', 'LOW-INCOME', 'HOMELESS']
        for demo in special_demos:
            demo_mask = layout_df['Metric'].str.match(f"{re.escape(demo)} {re.escape(enrollment_key)} %")
            if demo_mask.sum() > 0:
                layout_df.loc[demo_mask, 'Demographic'] = demo
                layout_df.loc[demo_mask, 'Metric'] = enrollment_key + ' ENROLLMENT'

        layout_df.loc[layout_df['Metric'] == enrollment_key + ' TOTAL ENROLLMENT', 'Metric'] = enrollment_key + ' ENROLLMENT'
    return layout_df

In [4]:
layout = {}

for year in range(2008,2018):
    s = "{:02d}".format(year - 2000)
    layout[year] = get_layout_file(s).iloc[:,[0,2,5]]
    layout[year] = layout[year][pd.to_numeric(layout[year][0], errors='coerce').notnull()].loc[:,[2,5]].reset_index(drop=True)
    layout[year] = layout[year].rename(columns={2:"Demographic",5:"Metric"})
    layout[year] = fix_enrollment(layout[year])
    layout[year]["Demographic"] = layout[year]['Demographic'].replace(demographic_key)
    mask = ~(layout[year]["Demographic"].isnull()) & (layout[year]["Demographic"] != "ALL")
    layout[year].loc[mask,"Metric"] = layout[year].loc[mask,"Metric"].astype(str) + " - " + layout[year].loc[mask,"Demographic"].astype(str)

In [5]:
report_card = {}

for key in filename_crosswalk.keys():
    if key > 2017:
        report_card[key] = pd.read_excel("./data/" + filename_crosswalk[key], sheet_name="General", dtype='object')
    else:
        report_card[key] = pd.read_csv("./data/" + filename_crosswalk[key], sep=";", header=None, dtype='object').rename(columns=layout[key]['Metric'])

In [6]:
# This edit is necessary to match later demographic automation
report_card[2018] = report_card[2018].rename(columns={'Student Enrollment - Total' : 'Student Enrollment'})

In [7]:
crosswalk = pd.read_excel('Local Historic Crosswalk.xlsx', sheet_name='Name Crosswalk')
crosswalk.index = crosswalk['Year']
crosswalk = crosswalk.drop(columns='Year')

demo_info = pd.read_excel('Local Historic Crosswalk.xlsx', sheet_name='Details')
disagg_info = pd.Series(crosswalk.columns, index=crosswalk.columns)
disagg_info = pd.Series(disagg_info.apply(lambda col: pd.merge(crosswalk[col].reset_index(), demo_info, left_on=['Year',col], right_on=['Year','Metric'], how='left')['Disaggregated'].any()), index=crosswalk.columns, name='Disaggregated')
# True if index is ever disaggregated, false otherwise

In [8]:
# EXPLORATORY CELL
search_string = "teach".lower()

for year in range(2008,2018):
    print(year,list(layout[year].loc[layout[year]['Metric'].str.lower().str.contains(search_string),'Metric']))
    
for year in range(2018,2024):
    print(year,list(report_card[year].columns[report_card[year].columns.str.lower().str.contains(search_string)]))

2008 ['% WHITE TEACH - DISTRICT', '% WHITE TEACH - STATE', '% BLACK TEACH - DISTRICT', '% BLACK TEACH - STATE', '% HISPANIC TEACH - DISTRICT', '% HISPANIC TEACH - STATE', '% ASIAN TEACH - DISTRICT', '% ASIAN TEACH - STATE', '% NATIVE AMER TEACH - DISTRICT', '% NATIVE AMER TEACH - STATE', '% MALE TEACH - DISTRICT', '% MALE TEACH - STATE', '% FEMALE TEACH - DISTRICT', '% FEMALE TEACH - STATE', 'TOTAL TEACH FTE COUNT- DISTRICT', 'TOTAL TEACH FTE COUNT- STATE', 'AVG TEACH EXP. - DISTRICT', 'AVG TEACH EXP. - STATE', 'AVG TEACH EXP. - DISTRICT HIGH POVERTY', 'AVG TEACH EXP. - DISTRICT LOW POVERTY', 'AVG TEACH EXP. - STATE HIGH POVERTY', 'AVG TEACH EXP. - STATE LOW POVERTY', 'AVG TEACH EXP. - ALL(DIST TYPE)', 'AVG TEACH EXP. - ALL(DIST SIZE)', 'PUPIL-TEACHER RATIO -ELEM DISTRICT', 'PUPIL-TEACHER RATIO -ELEM STATE', 'PUPIL-TEACHER RATIO -ELEM ALL(DIST TYPE)', 'PUPIL-TEACHER RATIO -ELEM ALL(DIST SIZE)', 'PUPIL-TEACHER RATIO -HS DISTRICT', 'PUPIL-TEACHER RATIO -HS STATE', 'PUPIL-TEACHER RATIO -H

In [9]:
# EXPLORATORY CELL
col_idx = 6
year_flag = 1

for year in range(2023,2017,-1):
    print(report_card[year].columns[col_idx])

for year in range(2017,2011,-1):
    print(layout[year].loc[col_idx + 1,'Metric'])
    
for year in range(2011,2007,-1):
    print(layout[year].loc[col_idx,'Metric'])

District Type
District Type
District Type
District Type
District Type
District Type
DISTRICT TYPE CODE (0,1,2,9)
DISTRICT TYPE CODE (0,1,2,9)
DISTRICT TYPE CODE (0,1,2,9)
DISTRICT TYPE CODE (0,1,2,9)
DISTRICT TYPE CODE (0,1,2,9)
DISTRICT TYPE CODE (0,1,2,9)
DISTRICT TYPE CODE (0,1,2,9)
DISTRICT TYPE CODE (0,1,2,9)
DISTRICT TYPE CODE (0,1,2,9)
DISTRICT TYPE CODE (0,1,2,9)


In [10]:
def add_demo_columns(columns, disagg_data):
    out_columns = []
    for col in columns:
        out_columns.append(col)
        if disagg_data[col]:
            out_columns += list(map(lambda x: col + " - " + x, pd.Series(demographic_key.values()).unique()))
    return out_columns

In [11]:
disagg_info

RCDTS                      False
Type                       False
District Type              False
School Type                False
School Name                False
District Name              False
City                       False
County                     False
Student Enrollment          True
Student Attendance Rate     True
Chronic Truancy Rate       False
Chronic Absenteeism         True
Name: Disaggregated, dtype: bool

In [12]:
# Replace demographic designations in report cards with standardized ones
# This is only done to relevant columns
for year in range(2018,2024):
    for col in crosswalk.columns:
        metric = crosswalk.loc[year,col]
        if metric == '# Student Enrollment':
            metric = '% Student Enrollment'

        metric_demos = pd.Series(report_card[year].columns)[list(map(lambda x: x.startswith(metric), report_card[year].columns))]
        metric_demos.index = metric_demos
        if len(metric_demos) > 1:
            metric_demos = metric_demos.apply(lambda x: x.split(" - "))
            metric_demos = metric_demos[metric_demos.apply(len) > 1]
            metric_demos = metric_demos.apply(lambda x: x[0] + " - " + demographic_key[x[1]])
            
            report_card[year] = report_card[year].rename(columns=metric_demos)

In [13]:
# These demo combos are not present in any report cards
absent_demo_combos = ['Student Enrollment - Female', 'Student Enrollment - Male', 'Student Enrollment - Migrant', 'Student Attendance Rate - Homeless', 'Student Attendance Rate - Children with Disabilities', 'Chronic Absenteeism - Migrant', 'Chronic Absenteeism - Homeless']

columns = add_demo_columns(crosswalk.columns, disagg_info)
columns = [item for item in columns if item not in absent_demo_combos]
new_columns = [item for item in columns if item not in crosswalk.columns]
demo_crosswalk = crosswalk.copy()
demo_crosswalk[new_columns] = np.nan

for col in new_columns:
    metric, demo = col.split(' - ')
    demo_crosswalk[col] = demo_crosswalk[metric] + " - " + demo
    if metric == 'Student Enrollment':
        demo_crosswalk.loc[range(2019,2024),col] = '% Student Enrollment - ' + demo

master_data = pd.DataFrame(columns=['Year'] + columns)

datasets = {}

In [15]:
def scope_data(rename_vals, year, crosswalk, scope, demo_crosswalk, report_card, dropped_district_columns):
    scoped_rename_vals = rename_vals.drop(crosswalk.loc[year,['School Name','School Type']])
    scoped_rename_vals.index = scoped_rename_vals.index.str.replace('SCHOOL', scope)
    scoped_rename_vals.index = [crosswalk.loc[year,'RCDTS']] + list(scoped_rename_vals.index[1:])
    
    found_columns = [item.replace('SCHOOL',scope) for item in demo_crosswalk.loc[year].dropna() if item.replace('SCHOOL',scope) in report_card[year].columns]
    found_columns.remove("DISTRICT TYPE NAME")
    found_columns.remove("DISTRICT NAME")
    found_columns = [crosswalk.loc[year,'RCDTS'],'SCHOOL TYPE NAME'] + found_columns

    scoped_data = report_card[year].loc[:,found_columns]
    dropped_district_columns[year] = list(set(scoped_rename_vals.index) - set(found_columns))
    dropped_district_columns[year].sort()
    scoped_data = scoped_data.rename(columns=scoped_rename_vals)

    return scoped_data, dropped_district_columns

In [16]:
def adjust_typing(data):
    for col in data.columns[8:]:
        try:
            data[col] = pd.to_numeric(data[col])
        except:
            data[col] = data[col].str.replace(",","")
            data[col] = data[col].str.strip()
            data[col] = pd.to_numeric(data[col])
    return data

In [17]:
dropped_columns = {}
dropped_district_columns = {}
dropped_state_columns = {}

for year in range(2008,2024):
    # the dropna here drops the colunns that are not included in the crosswalk
    # and thus not included in the report card for this year.
    years_columns = demo_crosswalk.loc[year].dropna() 
    rename_vals = pd.Series(years_columns.index.values, index=years_columns) # swap index and vals for renaming
    # this list comprehension drops any columns that are not found in the report card this year
    # this should drop demographic columns that are not found in this year, such as homeless enrollment 
    # counts for 2008 it should not however, drop columns that should be found in the report card.
    # Because it is capable of dropping columns that should be there, the dropped columns are recorded in 
    # dropped_columns to be checked later
    found_columns = [item for item in demo_crosswalk.loc[year].dropna() if item in report_card[year].columns]
    data = report_card[year].loc[:,found_columns]
    dropped_columns[year] = list(set(years_columns) - set(found_columns))
    dropped_columns[year].sort()
    data = data.rename(columns=rename_vals)

    # District and state level aggregation for 2008-2017
    if year < 2018:
        data['Type'] = 'School'

        district_data, dropped_district_columns = scope_data(rename_vals, year, crosswalk, 'DISTRICT', demo_crosswalk, report_card, dropped_district_columns)
        district_data['RCD'] = district_data['RCDTS'].str[:9]
        district_data = district_data[district_data['SCHOOL TYPE NAME'] != 'CHARTER SCH'].drop(columns='SCHOOL TYPE NAME')
        district_data['RCDTS'] = district_data['RCD'] + '000000'
        district_data = district_data.groupby('RCD').max().reset_index(drop=True)
        district_data['Type'] = 'District'
        
        

        state_data, dropped_state_columns = scope_data(rename_vals, year, crosswalk, 'STATE', demo_crosswalk, report_card, dropped_state_columns)
        state_data = pd.DataFrame(state_data.max()).T.drop(columns=['RCDTS','SCHOOL TYPE NAME','City','County'])
        state_data['Type'] = 'Statewide'

        data = pd.concat((data, district_data, state_data),axis=0,ignore_index=True)
        data = data[['RCDTS','Type'] + [item for item in data.columns if item not in ['RCDTS','Type']]]

    data = adjust_typing(data)
    data['Year'] = year
    datasets[year] = data

In [18]:
display(dropped_district_columns)
dropped_columns

{2008: ['ATTENDANCE RATE DISTRICT % - Native Hawaiian or Other Pacific Islander',
  'DISTRICT ENROLLMENT - Children with Disabilities',
  'DISTRICT ENROLLMENT - Homeless',
  'DISTRICT ENROLLMENT - IEP',
  'DISTRICT ENROLLMENT - Native Hawaiian or Other Pacific Islander'],
 2009: ['ATTENDANCE RATE DISTRICT % - Native Hawaiian or Other Pacific Islander',
  'DISTRICT ENROLLMENT - Children with Disabilities',
  'DISTRICT ENROLLMENT - Homeless',
  'DISTRICT ENROLLMENT - IEP',
  'DISTRICT ENROLLMENT - Native Hawaiian or Other Pacific Islander'],
 2010: ['ATTENDANCE RATE DISTRICT % - Native Hawaiian or Other Pacific Islander',
  'DISTRICT ENROLLMENT - Children with Disabilities',
  'DISTRICT ENROLLMENT - Homeless',
  'DISTRICT ENROLLMENT - Native Hawaiian or Other Pacific Islander'],
 2011: ['DISTRICT ENROLLMENT - Children with Disabilities',
  'DISTRICT ENROLLMENT - Homeless'],
 2012: ['DISTRICT ENROLLMENT - Children with Disabilities',
  'DISTRICT ENROLLMENT - Homeless'],
 2013: ['DISTRICT 

{2008: ['ATTENDANCE RATE SCHOOL % - Native Hawaiian or Other Pacific Islander',
  'SCHOOL ENROLLMENT - Children with Disabilities',
  'SCHOOL ENROLLMENT - Homeless',
  'SCHOOL ENROLLMENT - IEP',
  'SCHOOL ENROLLMENT - Native Hawaiian or Other Pacific Islander'],
 2009: ['ATTENDANCE RATE SCHOOL % - Native Hawaiian or Other Pacific Islander',
  'SCHOOL ENROLLMENT - Children with Disabilities',
  'SCHOOL ENROLLMENT - Homeless',
  'SCHOOL ENROLLMENT - IEP',
  'SCHOOL ENROLLMENT - Native Hawaiian or Other Pacific Islander'],
 2010: ['ATTENDANCE RATE SCHOOL % - Native Hawaiian or Other Pacific Islander',
  'SCHOOL ENROLLMENT - Children with Disabilities',
  'SCHOOL ENROLLMENT - Homeless',
  'SCHOOL ENROLLMENT - Native Hawaiian or Other Pacific Islander'],
 2011: ['SCHOOL ENROLLMENT - Children with Disabilities',
  'SCHOOL ENROLLMENT - Homeless'],
 2012: ['SCHOOL ENROLLMENT - Children with Disabilities',
  'SCHOOL ENROLLMENT - Homeless'],
 2013: ['SCHOOL ENROLLMENT - Children with Disabilitie

In [19]:
master_data = pd.concat(datasets.values(), ignore_index=True)
master_data = master_data.loc[:, ['Year'] + columns]
master_data = master_data.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)

In [21]:
master_data.to_excel('Historic Data.xlsx',index=False)
master_data.query("Type == 'District'").to_excel('Historic District Data.xlsx',index=False)